In [1]:
import cv2 as cv
import numpy as np
from threading import Thread

In [3]:

class VideoStream:
    def __init__(self, src):
        self.stream = cv.VideoCapture(src)
        self.stopped = False
        self.frame = None

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self  # So you can do .start().read()

    def update(self):
        while not self.stopped:
            if self.stream.isOpened():
                ret, frame = self.stream.read()
                if ret:
                    self.frame = frame

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()


In [4]:
# Capturing video frame from both phone cameras 

cap1 = VideoStream('http://192.168.0.177:4747/video').start()  # Left view
cap2 = VideoStream('http://192.168.0.114:4747/video').start() # Right view

#Process frames function => INcludes resizing, converting to grayscale, and reducing noise by blurring

def process_frames(frame):
    frame = cv.resize(frame, (640, 480))  # Resize to a standard size
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)  # Convert to grayscale
    return gray_frame


In [5]:


# Background subtractors
bg_sub1 = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
bg_sub2 = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

frame_count = 0  # for warming up background model

while True:
    frame1 = cap1.read()
    frame2 = cap2.read()

    if frame1 is None or frame2 is None:
        print("Waiting for frames...")
        continue

    frame1 = process_frames(frame1)
    frame2 = process_frames(frame2)

    frame_count += 1
    if frame_count < 30:  # allow background subtractor to warm up
        bg_sub1.apply(frame1)
        bg_sub2.apply(frame2)
        continue

    # Get foreground masks
    fg_mask1 = bg_sub1.apply(frame1)
    fg_mask2 = bg_sub2.apply(frame2)

    # Clean up noise in masks
    fg_mask1 = cv.medianBlur(fg_mask1, 5)
    fg_mask2 = cv.medianBlur(fg_mask2, 5)

    # Apply mask to get foreground objects
    fg1 = cv.bitwise_and(frame1, frame1, mask=fg_mask1)
    fg2 = cv.bitwise_and(frame2, frame2, mask=fg_mask2)

    # Convert to grayscale
    gray1 = cv.cvtColor(fg1, cv.COLOR_BGR2GRAY) if len(fg1.shape) == 3 else fg1
    gray2 = cv.cvtColor(fg2, cv.COLOR_BGR2GRAY) if len(fg2.shape) == 3 else fg2

    # Check for motion activity
    active1 = np.count_nonzero(fg_mask1) > 1000
    active2 = np.count_nonzero(fg_mask2) > 1000

    display = frame2.copy()

    if active1 and active2:
        # Compute absolute difference and threshold
        diff = cv.absdiff(gray1, gray2)
        _, thresh = cv.threshold(diff, 30, 255, cv.THRESH_BINARY)

        # Find and draw contours
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        cv.drawContours(display, contours, -1, (0, 0, 255), 2)

        # Similarity scoring
        nonzero = np.count_nonzero(thresh)
        total = thresh.shape[0] * thresh.shape[1]
        score = 100 - (nonzero / total * 100)
        cv.putText(display, f"Similarity: {score:.2f}%", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if score < 80:
            cv.putText(display, "Movement Not Synchronized!", (10, 460),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    else:
        # No significant motion in one or both
        cv.putText(display, "No significant motion in one or both feeds!", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Show results
    cv.imshow("Left View", frame1)
    cv.imshow("Right View", frame2)
    cv.imshow("Difference Highlighted", display)
    cv.imshow("Threshold Map", thresh if active1 and active2 else np.zeros_like(gray1))

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap1.stop()
cap2.stop()
cv.destroyAllWindows()


Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_30148\3743077464.py", line 14, in update
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_30148\3743077464.py", line 14, in update
cv2.error: Unknown C++ exception from OpenCV code
cv2.error: Unknown C++ exception from OpenCV code


In [6]:
import cv2 as cv
import numpy as np
from threading import Thread
from skimage.metrics import structural_similarity as ssim

# === Fingertip Color Range in HSV (Adjust as needed) ===
lower_color = np.array([0, 150, 100])     # Red hue lower bound
upper_color = np.array([10, 255, 255])    # Red hue upper bound

# === Canvas Size ===
WIDTH, HEIGHT = 640, 480

# === VideoStream Class for Better Performance ===
class VideoStream:
    def __init__(self, src):
        self.stream = cv.VideoCapture(src)
        self.stopped = False
        self.frame = None

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while not self.stopped:
            if self.stream.isOpened():
                ret, frame = self.stream.read()
                if ret:
                    self.frame = frame

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()

# === Initialize Streams ===
vs1 = VideoStream('http://192.168.0.177:4747/video').start()  # Back view
vs2 = VideoStream('http://192.168.0.114:4747/video').start() # Front view
canvas1 = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
canvas2 = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)

last_point1, last_point2 = None, None

def detect_finger(frame):
    if frame is None:
        return None
    frame = cv.flip(frame, 1)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, lower_color, upper_color)
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv.contourArea)
        if cv.contourArea(largest) > 300:
            M = cv.moments(largest)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                return (cx, cy)
    return None

def update_canvas(canvas, point, last_point):
    if point and last_point:
        cv.line(canvas, last_point, point, (255, 255, 255), 4)

def compare_images(img1, img2):
    gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    gray1 = cv.resize(gray1, (256, 256))
    gray2 = cv.resize(gray2, (256, 256))
    score, diff = ssim(gray1, gray2, full=True)
    diff = (diff * 255).astype("uint8")
    return score, cv.cvtColor(diff, cv.COLOR_GRAY2BGR)

print("Press 'q' to quit. Press 'r' to reset canvases.")

while True:
    frame1 = vs1.read()
    frame2 = vs2.read()

    if frame1 is None or frame2 is None:
        continue

    point1 = detect_finger(frame1)
    point2 = detect_finger(frame2)

    update_canvas(canvas1, point1, last_point1)
    update_canvas(canvas2, point2, last_point2)

    last_point1 = point1
    last_point2 = point2

    score, diff_canvas = compare_images(canvas1, canvas2)
    cv.putText(diff_canvas, f"Score: {score:.2f}", (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv.imshow("Back Drawing", canvas1)
    cv.imshow("Front Drawing", canvas2)
    cv.imshow("Difference", diff_canvas)

    key = cv.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        canvas1[:] = 0
        canvas2[:] = 0
        last_point1 = last_point2 = None

vs1.stop()
vs2.stop()
cv.destroyAllWindows()


Press 'q' to quit. Press 'r' to reset canvases.


Exception in thread Thread-15:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-16:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_48148\596969375.py", line 27, in update
cv2.error: Unknown C++ exception from OpenCV code
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_48148\596969375.py", line 27, in update
cv2.error: Unknown C++ exception from OpenCV code


In [ ]:
import cv2 as cv
import numpy as np
from threading import Thread
import mediapipe as mp

# === Canvas Settings ===
WIDTH, HEIGHT = 640, 480
DRAW_COLOR = (0, 0, 0)  # Black drawing on white background
LINE_THICKNESS = 8

# === VideoStream Class ===
class VideoStream:
    def __init__(self, src):
        self.stream = cv.VideoCapture(src)
        self.stopped = False
        self.frame = None

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while not self.stopped:
            if self.stream.isOpened():
                ret, frame = self.stream.read()
                if ret:
                    self.frame = cv.resize(frame, (WIDTH, HEIGHT))

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()

# === Initialize Streams ===
vs1 = VideoStream('http://192.168.0.177:4747/video').start()  # Back view
vs2 = VideoStream('http://192.168.0.114:4747/video').start()  # Front view

# Initialize white canvases
canvas1 = np.full((HEIGHT, WIDTH, 3), 255, dtype=np.uint8)
canvas2 = np.full((HEIGHT, WIDTH, 3), 255, dtype=np.uint8)
last_point1, last_point2 = None, None

# === MediaPipe Setup ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

def detect_fingertip(frame):
    if frame is None:
        return None
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        x = int(index_tip.x * WIDTH)
        y = int(index_tip.y * HEIGHT)
        cv.circle(frame, (x, y), 10, (0, 255, 0), -1)
        return (x, y)
    return None

def update_canvas(canvas, point, last_point):
    if point:
        cv.circle(canvas, point, LINE_THICKNESS//2, DRAW_COLOR, -1)
        if last_point and (point != last_point):
            cv.line(canvas, last_point, point, DRAW_COLOR, LINE_THICKNESS)

def compare_drawings(img1, img2):
    # Convert to grayscale and threshold
    gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    
    # Apply threshold to get binary images
    _, thresh1 = cv.threshold(gray1, 200, 255, cv.THRESH_BINARY_INV)
    _, thresh2 = cv.threshold(gray2, 200, 255, cv.THRESH_BINARY_INV)
    
    # Find contours for each drawing
    contours1, _ = cv.findContours(thresh1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv.findContours(thresh2, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # Create blank masks
    mask1 = np.zeros_like(thresh1)
    mask2 = np.zeros_like(thresh2)
    
    # Draw contours on masks
    cv.drawContours(mask1, contours1, -1, 255, thickness=cv.FILLED)
    cv.drawContours(mask2, contours2, -1, 255, thickness=cv.FILLED)
    
    # Calculate intersection and union
    intersection = cv.bitwise_and(mask1, mask2)
    union = cv.bitwise_or(mask1, mask2)
    
    # Calculate similarity
    intersection_area = cv.countNonZero(intersection)
    union_area = cv.countNonZero(union)
    
    if union_area == 0:
        return 0.0, np.zeros_like(img1)
    
    similarity = intersection_area / union_area
    
    # Create difference visualization
    diff = cv.bitwise_xor(mask1, mask2)
    diff_visual = cv.cvtColor(diff, cv.COLOR_GRAY2BGR)
    
    return similarity, diff_visual

# Main loop
while True:
    frame1 = vs1.read()
    frame2 = vs2.read()

    if frame1 is None or frame2 is None:
        continue

    point1 = detect_fingertip(frame1)
    point2 = detect_fingertip(frame2)

    if point1:
        update_canvas(canvas1, point1, last_point1)
        last_point1 = point1
    if point2:
        update_canvas(canvas2, point2, last_point2)
        last_point2 = point2

    # Compare drawings
    score, diff_vis = compare_drawings(canvas1, canvas2)
    
    # Display views
    cv.imshow("Back Camera", frame1)
    cv.imshow("Front Camera", frame2)
    cv.imshow("Back Drawing", canvas1)
    cv.imshow("Front Drawing", canvas2)
    
    # Show difference and score
    cv.putText(diff_vis, f"Similarity: {score:.2f}", (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv.imshow("Difference", diff_vis)

    key = cv.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        canvas1.fill(255)
        canvas2.fill(255)
        last_point1 = last_point2 = None

# Cleanup
vs1.stop()
vs2.stop()
hands.close()
cv.destroyAllWindows()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "c:\Users\khare\Anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_49316\4231596643.py", line 25, in update
    self._target(*self._args, **self._kwargs)
  File "C:\Users\khare\AppData\Local\Temp\ipykernel_49316\4231596643.py", line 25, in update
cv2.error: Unknown C++ exception from OpenCV code
cv2.error: Unknown C++ exception from OpenCV code


In [7]:
import mediapipe as mp
import cv2 as cv
import numpy as np
from mediapipe.tasks import python
from threading import Thread
from mediapipe.tasks.python import vision
import time

# === MediaPipe Hand Detection Setup ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

# === VideoStream Class for Better Performance ===

class VideoStream:
    def __init__(self, url):
        self.stream = cv.VideoCapture(url)
        self.stopped = False
        self.frame = None

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while not self.stopped:
            if self.stream.isOpened():
                ret, frame = self.stream.read()
                if ret:
                    self.frame = frame

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()

# === Initialize Video Streams ===
vs1 = VideoStream('http://192.168.0.177:4747/video').start()  # Back view
vs2 = VideoStream('http://192.168.0.114:4747/video').start()  # Front view




In [ ]:
import mediapipe as mp
import cv2 as cv
import numpy as np
from mediapipe.tasks import python
from threading import Thread
from mediapipe.tasks.python import vision
import time

# === MediaPipe Hand Detection Setup ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

# === VideoStream Class for Better Performance ===

class VideoStream:
    def __init__(self, url):
        self.stream = cv.VideoCapture(url)
        self.stopped = False
        self.frame = None

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while not self.stopped:
            if self.stream.isOpened():
                ret, frame = self.stream.read()
                if ret:
                    self.frame = frame

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()

# === Initialize Video Streams ===
vs1 = VideoStream('http://192.168.0.177:4747/video').start()  # Back view
vs2 = VideoStream('http://192.168.0.114:4747/video').start()  # Front view




def classify_pose(hand_landmarks):
    # Get key landmarks (using correct MediaPipe indices)
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]  # Middle joint
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    
    # Finger states (True = extended)
    index_extended = index_tip.y < index_mcp.y
    middle_extended = middle_tip.y < middle_mcp.y
    thumb_extended = thumb_tip.x < index_mcp.x  # Works for right hand
    
    # Pose classification
    if index_extended and middle_extended and not thumb_extended:
        return "VICTORY"  # ✌️
    elif not index_extended and not middle_extended:
        return "FIST"  # ✊
    elif index_extended and not middle_extended and thumb_extended:
        return "THUMBS_UP"  # 👍
    else:
        return "OPEN"  # ✋
    

# Create named windows first (for consistent sizing)
cv.namedWindow("Player 1 View", cv.WINDOW_NORMAL)
cv.namedWindow("Player 2 View", cv.WINDOW_NORMAL)
cv.namedWindow("Mirror Challenge Game", cv.WINDOW_NORMAL)

while True:
    frame1 = vs1.read()  # Player 1
    frame2 = vs2.read()  # Player 2
    
    # Process frames and get poses (your existing code)
    p1_pose = None
    p2_pose = None
    
    # Process Player 1
    if frame1 is not None:
        results1 = hands.process(cv.cvtColor(frame1, cv.COLOR_BGR2RGB))
        if results1.multi_hand_landmarks:
            p1_pose = classify_pose(results1.multi_hand_landmarks[0])
            # Draw landmarks on frame
            mp_drawing.draw_landmarks(
                frame1, results1.multi_hand_landmarks[0], 
                mp_hands.HAND_CONNECTIONS)
    
    # Process Player 2
    if frame2 is not None:
        results2 = hands.process(cv.cvtColor(frame2, cv.COLOR_BGR2RGB))
        if results2.multi_hand_landmarks:
            p2_pose = classify_pose(results2.multi_hand_landmarks[0])
            mp_drawing.draw_landmarks(
                frame2, results2.multi_hand_landmarks[0], 
                mp_hands.HAND_CONNECTIONS)
    
    # Create game output display
    output = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Add game info to output
    cv.putText(output, f"Player 1: {p1_pose or 'No pose'}", (50, 50), 
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv.putText(output, f"Player 2: {p2_pose or 'No pose'}", (50, 100), 
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv.putText(output, "Mirror the pose with 1s delay!", (50, 150), 
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
    
    # Display all windows
    if frame1 is not None:
        cv.imshow("Player 1 View", frame1)
    if frame2 is not None:
        cv.imshow("Player 2 View", frame2)
    cv.imshow("Mirror Challenge Game", output)
    
    # Exit on 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
vs1.stop()
vs2.stop()
hands.close()
cv.destroyAllWindows()

In [8]:
# Create named windows first (for consistent sizing)
cv.namedWindow("Player 1 View", cv.WINDOW_NORMAL)
cv.namedWindow("Player 2 View", cv.WINDOW_NORMAL)
cv.namedWindow("Mirror Challenge Game", cv.WINDOW_NORMAL)

while True:
    frame1 = vs1.read()  # Player 1
    frame2 = vs2.read()  # Player 2
    
    # Process frames and get poses (your existing code)
    p1_pose = None
    p2_pose = None
    
    # Process Player 1
    if frame1 is not None:
        results1 = hands.process(cv.cvtColor(frame1, cv.COLOR_BGR2RGB))
        if results1.multi_hand_landmarks:
            p1_pose = classify_pose(results1.multi_hand_landmarks[0])
            # Draw landmarks on frame
            mp_drawing.draw_landmarks(
                frame1, results1.multi_hand_landmarks[0], 
                mp_hands.HAND_CONNECTIONS)
    
    # Process Player 2
    if frame2 is not None:
        results2 = hands.process(cv.cvtColor(frame2, cv.COLOR_BGR2RGB))
        if results2.multi_hand_landmarks:
            p2_pose = classify_pose(results2.multi_hand_landmarks[0])
            mp_drawing.draw_landmarks(
                frame2, results2.multi_hand_landmarks[0], 
                mp_hands.HAND_CONNECTIONS)
    
    # Create game output display
    output = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Add game info to output
    cv.putText(output, f"Player 1: {p1_pose or 'No pose'}", (50, 50), 
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv.putText(output, f"Player 2: {p2_pose or 'No pose'}", (50, 100), 
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv.putText(output, "Mirror the pose with 1s delay!", (50, 150), 
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
    
    # Display all windows
    if frame1 is not None:
        cv.imshow("Player 1 View", frame1)
    if frame2 is not None:
        cv.imshow("Player 2 View", frame2)
    cv.imshow("Mirror Challenge Game", output)
    
    # Exit on 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
vs1.stop()
vs2.stop()
hands.close()
cv.destroyAllWindows()

KeyboardInterrupt: 